# PDF to CSV

In [1]:
SRC = 'https://www.kyoukaikenpo.or.jp/~/media/Files/shared/hokenryouritu/r5/ippan/r50213tokyo.pdf'
DST = '2023-04-01.csv'

In [2]:
import datetime
import hashlib
import io
import pathlib
import urllib.request

try:
    import tabula
except ModuleNotFoundError:
    !pip install tabula-py
    import tabula

datetime.datetime.now().astimezone(datetime.timezone.utc).isoformat()

'2023-12-19T05:06:02.549847+00:00'

In [3]:
data = io.BytesIO()

with urllib.request.urlopen(SRC) as f:
    data.write(f.read())

hashlib.sha256(data.getvalue()).hexdigest()

'f376d073d6950d3584d816c0f51cf5e40604135ad0392079f39427361be97ce9'

In [4]:
data.seek(0)

df = tabula.read_pdf(data, lattice=True, pages=1)[0]

df = df.iloc[:, [1, 2, 3, 4, 5, 6, 7]]

df.columns = """
標準報酬_等級
標準報酬_月額
報酬月額
介護保険の被保険者でない場合_全額
介護保険の被保険者でない場合_折半額
介護保険の被保険者の場合_全額
介護保険の被保険者の場合_折半額
""".strip().split()

df['標準報酬_等級'] = df['標準報酬_等級'].str.replace(r'\([0-9]+\)', '', regex=True)
df = df[~df['標準報酬_等級'].isnull()]
df = df[df['標準報酬_等級'].str.match(r'^[0-9]+$')]
df['標準報酬_等級'] = df['標準報酬_等級'].astype(int)
df = df.set_index('標準報酬_等級').sort_index()

assert df.shape[0] == 50, f'等級数 ({df.shape[0]}) に過不足があります。'

s = df['報酬月額'].str.replace(r'[^0-9~]', '', regex=True)
df_ = s.str.split('~', n=1, expand=True)
df_ = df_.rename(columns={0: '報酬月額_以上', 1: '報酬月額_未満'})
df = df.merge(df_, left_index=True, right_index=True)

del df['報酬月額']

for x in df.columns:
    df[x] = df[x].str.replace(',', '')

df = df.replace('', float('nan'))
df['報酬月額_以上'] = df['報酬月額_以上'].astype(float)
df['報酬月額_未満'] = df['報酬月額_未満'].astype(float)
df['標準報酬_月額'] = df['標準報酬_月額'].astype(int)

for x in """
介護保険の被保険者でない場合_全額
介護保険の被保険者でない場合_折半額
介護保険の被保険者の場合_全額
介護保険の被保険者の場合_折半額
""".strip().split():
    df[x] = df[x].astype(float)

for x in """
介護保険の被保険者でない場合_
介護保険の被保険者の場合_
""".strip().split():
    df[x + '控除額'] = (df[x + '折半額'] + .49).astype(int)

df = df["""
標準報酬_月額
報酬月額_以上
報酬月額_未満
介護保険の被保険者でない場合_全額
介護保険の被保険者でない場合_折半額
介護保険の被保険者でない場合_控除額
介護保険の被保険者の場合_全額
介護保険の被保険者の場合_折半額
介護保険の被保険者の場合_控除額
""".strip().split()]

df.to_csv(DST)

hashlib.sha256(pathlib.Path(DST).read_bytes()).hexdigest()

Error importing jpype dependencies. Fallback to subprocess.
No module named 'jpype'
Got stderr: 12月 19, 2023 2:06:03 午後 org.apache.fontbox.ttf.CmapSubtable processSubtype14
警告: Format 14 cmap table is not supported and will be ignored
12月 19, 2023 2:06:03 午後 org.apache.fontbox.ttf.CmapSubtable processSubtype14
警告: Format 14 cmap table is not supported and will be ignored



'11701df9243c6792d31a1c2f3950b8a2ee5ab42bbef661080f3cc466fb30910d'